In [3]:
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math

import requests
from bs4 import BeautifulSoup
import wget
import os

from scrape import *
from get_metadata import *
import wget

In [4]:
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from wordcloud import WordCloud, STOPWORDS

### Word cloud function

In [65]:
"""
Masked wordcloud
================
Using a mask you can generate wordclouds in arbitrary shapes.
"""

def get_word_cloud(text, mask_image, output_dir, filename):

    # get data directory (using getcwd() is needed to support running example in generated IPython notebook)
    #d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
    d = output_dir

    # Read the whole text.
    #text = open(path.join(d, 'alice.txt')).read()

    # read the mask image
    # taken from
    # http://www.stencilry.org/stencils/movies/alice%20in%20wonderland/255fk.jpg
    #alice_mask = np.array(Image.open(path.join(d, "alice_mask.png")))
    mask = np.array(Image.open(mask_image))

    stopwords = set(STOPWORDS)
    stopwords.add("said", "thank")

    wc = WordCloud(background_color="white", max_words=2000, mask=mask,
                   stopwords=stopwords, contour_width=3, contour_color='steelblue')

    # generate word cloud
    wc.generate(text)

    # store to file
    wc.to_file(path.join(d, filename))

    return wc

### Circle pack data

In [5]:
base_path = "/Users/nesara/Documents/aim/git/articles/twitter_analysis/code/data/accounts/"

In [18]:
top_accounts = ["katyperry", "justinbieber", "BarackObama", "rihanna", "taylorswift13", "ladygaga", "TheEllenShow", "Cristiano", "YouTube", "jtimberlake", "ArianaGrande", "KimKardashian", "realDonaldTrump", "selenagomez", "britneyspears", "twitter", "cnnbrk", "shakira", "jimmyfallon", "BillGates", "narendramodi", "JLo", "nytimes", "neymarjr", "BrunoMars", "KingJames", "Oprah", "MileyCyrus", "CNN", "NiallOfficial", "BBCBreaking", "Drake", "iamsrk", "instagram", "SrBachchan", "BeingSalmanKhan", "SportsCenter", "KevinHart4real", "LilTunechi", "espn", "wizkhalifa", "Harry_Styles", "Louis_Tomlinson", "LiamPayne", "Pink", "realmadrid", "onedirection", "NASA", "aliciakeys", "chrisbrown"]
top_accounts

['katyperry',
 'justinbieber',
 'BarackObama',
 'rihanna',
 'taylorswift13',
 'ladygaga',
 'TheEllenShow',
 'Cristiano',
 'YouTube',
 'jtimberlake',
 'ArianaGrande',
 'KimKardashian',
 'realDonaldTrump',
 'selenagomez',
 'britneyspears',
 'twitter',
 'cnnbrk',
 'shakira',
 'jimmyfallon',
 'BillGates',
 'narendramodi',
 'JLo',
 'nytimes',
 'neymarjr',
 'BrunoMars',
 'KingJames',
 'Oprah',
 'MileyCyrus',
 'CNN',
 'NiallOfficial',
 'BBCBreaking',
 'Drake',
 'iamsrk',
 'instagram',
 'SrBachchan',
 'BeingSalmanKhan',
 'SportsCenter',
 'KevinHart4real',
 'LilTunechi',
 'espn',
 'wizkhalifa',
 'Harry_Styles',
 'Louis_Tomlinson',
 'LiamPayne',
 'Pink',
 'realmadrid',
 'onedirection',
 'NASA',
 'aliciakeys',
 'chrisbrown']

#### Number of followers

In [3]:
dfp = pd.DataFrame()
idx = 0

for account in os.listdir(base_path):
    if not account.startswith("."):
        profile_file = base_path + account + "/" + account+"_profile.json"
        with open(profile_file) as fp:
            profile = json.load(fp)
            
        if len(profile)>0:
            profile = profile[0]
            dfp.at[idx, "name"] = profile["name"]
            dfp.at[idx, "screen_name"] = profile["screen_name"]
            dfp.at[idx, "followers_count"] = profile["followers_count"]
            dfp.at[idx, "friends_count"] = profile["friends_count"]
            dfp.at[idx, "favourites_count"] = profile["favourites_count"]
            dfp.at[idx, "statuses_count"] = profile["statuses_count"]
            
            # Get url of profile image
            entire_json_file = base_path + account + "/" + account+".json"
            with open(entire_json_file) as fp:
                entire_json = json.load(fp)
            dfp.at[idx, "image"] = entire_json[0]["user"]["profile_image_url_https"]
            """
            image_dir = "../code/data/profile_images/" + account + "_image.jpg"
            try:
                wget.download(entire_json[0]["user"]["profile_image_url_https"], out=image_dir)
            except:
                print("No image ..." + account)
            """
            idx += 1
        else:
            print("No tweets ..." + account)
            
# Set account type: # 0:Individual 1:Organisation
dfp["account_type"] = 0
organisation_list = ["Twitter", "realmadrid", "YouTube", "BBCBreaking", "CNN", "nytimes", "SportsCenter",
                     "espn", "instagram", "NASA"]
dfp.loc[dfp["screen_name"].isin(organisation_list), "account_type"] = 1

No tweets ...taylorswift13
No tweets ...onedirection
No tweets ...cnnbrk


In [4]:
dfp.head(2)

,name,screen_name,followers_count,friends_count,favourites_count,statuses_count,image,account_type
0,Amitabh Bachchan,SrBachchan,36102476.0,1860.0,68.0,63753.0,https://pbs.twimg.com/profile_images/768780297...,0
1,Twitter,Twitter,55900683.0,140.0,5608.0,8824.0,https://pbs.twimg.com/profile_images/101379824...,1


#### Save circle pack data in csv file

In [62]:
dfp_followers = dfp[["screen_name", "followers_count", "name", "image", "account_type"]].copy()
dfp_followers.columns = ["id", "value", "name", "image", "type"]
dfp_followers["value"] = dfp_followers["value"].astype(int)
dfp_followers.to_csv("../code/data/circle_pack_followers.csv", index=False)

dfp_friends = dfp[["screen_name", "friends_count", "name", "image", "account_type"]].copy()
dfp_friends.columns = ["id", "value", "name", "image", "type"]
dfp_friends["value"] = dfp_friends["value"].astype(int)
dfp_friends.to_csv("../code/data/circle_pack_friends.csv", index=False)

dfp_favourites = dfp[["screen_name", "favourites_count", "name", "image", "account_type"]].copy()
dfp_favourites.columns = ["id", "value", "name", "image", "type"]
dfp_favourites["value"] = dfp_favourites["value"].astype(int)
dfp_favourites.to_csv("../code/data/circle_pack_favourites.csv", index=False)

dfp_status = dfp[["screen_name", "statuses_count", "name", "image", "account_type"]].copy()
dfp_status.columns = ["id", "value", "name", "image", "type"]
dfp_status["value"] = dfp_status["value"].astype(int)
dfp_status.to_csv("../code/data/circle_pack_status.csv", index=False)

### Most liked/retweeted tweets

In [12]:
dfl = pd.DataFrame()
dfall = pd.DataFrame()

for account in os.listdir(base_path):
    if not account.startswith("."):
        profile_file = base_path + account + "/" + account+"_profile.json"
        with open(profile_file) as fp:
            profile = json.load(fp)
            
        if len(profile)>0:
            profile = profile[0]
            
            csv_file = base_path + account + "/" + account+".csv"
            df = pd.read_csv(csv_file)
            df["screen_name"] = profile["screen_name"]
            df["name"] = profile["name"]
            df["followers_count"] = profile["followers_count"]
            dfl = dfl.append(df)

            dfil = df[["favorite_count","text","created_at",
                       "screen_name","name", "followers_count"]].copy()
            dfil = dfil.sort_values("favorite_count", ascending=False).head(5)
            #dfil = dfil[["screen_name", "favorite_count", "text"]]
            #dfil.columns = ["id", "value", "tweet"]
            dfil = dfil[["screen_name", "favorite_count", "text", "created_at", "followers_count", "name"]]
            dfil.columns = ["id", "likes", "tweet", "created_at", "followers_count", "name"]
            dfil["type"] = 0
            dfall = dfall.append(dfil)
            dfil.to_csv("../code/data/most_liked/"+account+".csv", index=False)

dftop = dfl.sort_values("favorite_count", ascending=False).head(5)
#dftop = dftop[["screen_name", "favorite_count", "text"]].copy()
#dftop.columns = ["id", "value", "tweet"]
dftop = dftop[["screen_name", "favorite_count", "text", "created_at", "followers_count", "name"]].copy()
dftop.columns = ["id", "likes", "tweet", "created_at", "followers_count", "name"]
dftop["type"] = 1
dfall = dfall.append(dftop)
dftop.to_csv("../code/data/most_liked/"+"All"+".csv", index=False)

"""
dfall["account_type"] = 0
organisation_list = ["Twitter", "realmadrid", "YouTube", "BBCBreaking", "CNN", "nytimes", "SportsCenter",
                     "espn", "instagram", "NASA"]
dfall.loc[dfall["id"].isin(organisation_list), "account_type"] = 1

dfall.to_csv("../code/data/most_liked_tweets.csv", index=False)
"""

'\ndfall["account_type"] = 0\norganisation_list = ["Twitter", "realmadrid", "YouTube", "BBCBreaking", "CNN", "nytimes", "SportsCenter",\n                     "espn", "instagram", "NASA"]\ndfall.loc[dfall["id"].isin(organisation_list), "account_type"] = 1\n\ndfall.to_csv("../code/data/most_liked_tweets.csv", index=False)\n'

#### Highest difference in top 5 most liked tweets 

In [146]:
dfd = pd.DataFrame()
idx = 0
for acc in dfall["id"].unique():
    diff = dfall[dfall["id"]==acc].head(1)["value"].values[0] - dfall[dfall["id"]==acc].tail(1)["value"].values[0]
    dfd.at[idx, "id"] = acc
    dfd.at[idx, "diff"] = diff
    idx += 1

In [147]:
dfd.sort_values("diff", ascending=False)

,id,diff
25,BarackObama,646383.0
9,justinbieber,441437.0
36,ddlovato,349753.0
15,YouTube,268931.0
7,ladygaga,257300.0
35,ArianaGrande,150676.0
31,rihanna,149122.0
23,Louis_Tomlinson,138237.0
32,KimKardashian,111603.0
45,NiallOfficial,110667.0


### Word cloud

In [64]:
dfl = pd.DataFrame()
dfall = pd.DataFrame()
output_dir = "../code/data/word_cloud/"
word_cloud_mask = "../code/data/twitter_mask.png"

for account in ["BillGates"]: #os.listdir(base_path):
    if not account.startswith("."):
        profile_file = base_path + account + "/" + account+"_profile.json"
        with open(profile_file) as fp:
            profile = json.load(fp)
            
        if len(profile)>0:
            profile = profile[0]
            
            csv_file = base_path + account + "/" + account+".csv"
            df = pd.read_csv(csv_file)
            df["screen_name"] = profile["screen_name"]
            df["name"] = profile["name"]
            df["followers_count"] = profile["followers_count"]

            tweet_text = ''
            for i in range(len(df)):
                tweet_text += df.iloc[i]["text"].split("http")[0]
            tweet_text = tweet_text.replace("\n", "")
            
            if len(tweet_text) > 10:
                filename = account + ".png"
                wc = get_word_cloud(tweet_text, word_cloud_mask, output_dir, filename)

### Hierarchial edge bundling

In [46]:
dfc = pd.DataFrame(columns={"from", "to", "num_replies"})
#dff = pd.DataFrame()
idx = 0
json_data = []

for account in os.listdir(base_path):
    if not account.startswith("."):
        profile_file = base_path + account + "/" + account+"_profile.json"
        with open(profile_file) as fp:
            profile = json.load(fp)
            
        if len(profile)>0:
            profile = profile[0]
            
            csv_file = base_path + account + "/" + account+".csv"
            df = pd.read_csv(csv_file)
            df["screen_name"] = profile["screen_name"]
            df["name"] = profile["name"]
            df["followers_count"] = profile["followers_count"]
            #dfl = dfl.append(df)
            
            repliedto_arr = []
            for i in range(len(df)):
                for word in df.iloc[i]["text"].split(" "):
                    if word.startswith("@"):
                        repliedto = word.split("@")[1]
                        if repliedto in top_accounts:
                            
                            dfe = dfc[(dfc["from"]==account) & (dfc["to"]==repliedto)]
                            if len(dfe)>0:
                                num_prev_replies = dfe["num_replies"].values[0]
                                dfc.loc[(dfc["from"]==account) & (dfc["to"]==repliedto), "num_replies"] = num_prev_replies + 1
                            else:
                                num_prev_replies = 0
                                dfc.at[idx, "from"] = account
                                dfc.at[idx, "to"] = repliedto
                                dfc.at[idx, "num_replies"] = 1
                                repliedto_arr.append("flare." + repliedto)
                            idx += 1
            
            #dff.at[idx, "from"] = "flare." + account
            #dff.at[idx, "to"] = repliedto_arr
            #dff.at[idx, "size"] = num_prev_replies + 1
            
            dff = {
                    "name": "flare."+account,
                    "size": num_prev_replies + 1,
                    "imports": repliedto_arr
                  }
            json_data.append(dff)
                            
dfc = dfc[["from", "to", "num_replies"]]
dfc

with open("../code/data/link.json", "w") as fp:
    json.dump(json_data, fp)

In [21]:
for word in dfl.iloc[-10]["text"].split(" "):
    if word.startswith("@"):
        repliedto = word.split("@")[1]
        if repliedto in top_accounts:
            print(repliedto)

In [17]:
account

'NASA'

In [20]:
repliedto in top_accounts

False